
## REPRODUCING TABLE 1: Predicting GDP per capita
### Paper: "Multidimensional tie strength and economic development"

### Final approach and results ###
The code below shows the result that better match the Table 1 from the paper. We used the pre-computed Diversity Measures of the regression_file.csv dataset. The dataset contained multiple versions of each diversity metric with different configurations:

- diversity_spatial_all_minstrength_X (where X = 1, 2, 3, 4, 5)
- spatial_diversity_[dimension]_[threshold] (e.g., spatial_diversity_knowledge_0.99)

Therefore, we systematically tested different column combinations, to finally check that by using the pre-computed diversity columns with minstrength_5, we achieved the closest match to the paper's published results.
 

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson

# Load data
df = pd.read_csv('data/regression_file.csv')
dim_df = pd.read_csv('data/reddit_messages_dimensions.csv', sep="|")

# Calculate population density
state_areas = {
    "AL": 131170, "AK": 1723337, "AZ": 295234, "AR": 137732,
    "CA": 423967, "CO": 269601, "CT": 14357, "DE": 6446,
    "FL": 170312, "GA": 153910, "HI": 28313, "ID": 216443,
    "IL": 149995, "IN": 94326, "IA": 145746, "KS": 213100,
    "KY": 104656, "LA": 135659, "ME": 91633, "MD": 32131,
    "MA": 27336, "MI": 250487, "MN": 225163, "MS": 125438,
    "MO": 180540, "MT": 380831, "NE": 200330, "NV": 286380,
    "NH": 24214, "NJ": 22591, "NM": 314917, "NY": 141297,
    "NC": 139391, "ND": 183108, "OH": 116098, "OK": 181037,
    "OR": 254806, "PA": 119280, "RI": 4001, "SC": 82933,
    "SD": 199729, "TN": 109153, "TX": 695662, "UT": 219882,
    "VT": 24906, "VA": 110787, "WA": 184661, "WV": 62756,
    "WI": 169635, "WY": 253335, "DC": 177
}

df["population_density"] = df["population_2019"] / df["state_code"].map(state_areas)

# Filter to 44 states (exclude DC, AK, MS, WV, SD, WY, ND)
excluded = ["DC", "AK", "MS", "WV", "SD", "WY", "ND"]
df = df[~df["state_code"].isin(excluded)].copy()

print(f"Sample size: {len(df)} states")
print(f"Excluded states: {', '.join(excluded)}\n")

Sample size: 44 states
Excluded states: DC, AK, MS, WV, SD, WY, ND



In [2]:
# Select variables (best configuration: minstrength_5)
data = df[[
    'gdp per capita (2017)',
    'population_density',
    'diversity_spatial_all_minstrength_5',
    'spatial_diversity_knowledge_0.99',
    'spatial_diversity_support_0.99'
]].dropna()

# Standardize variables
def standardize(x):
    return (x - x.mean()) / x.std()

y = standardize(data['gdp per capita (2017)'])
x1 = standardize(data['population_density'])
x2 = standardize(data['diversity_spatial_all_minstrength_5'])
x3 = standardize(data['spatial_diversity_knowledge_0.99'])
x4 = standardize(data['spatial_diversity_support_0.99'])

# MODEL 1: Population density only
X1 = sm.add_constant(x1)
model1 = sm.OLS(y, X1).fit()

# MODEL 2: Full communication graph
X2 = sm.add_constant(pd.DataFrame({'pop_density': x1, 'D_spatial': x2}))
model2 = sm.OLS(y, X2).fit()

# MODEL 3: Dimension-specific graphs
X3 = sm.add_constant(pd.DataFrame({
    'pop_density': x1, 
    'D_knowledge': x3, 
    'D_support': x4
}))
model3 = sm.OLS(y, X3).fit()

#Results summary:

print("="*80)
print("TABLE 1 REPRODUCTION RESULTS")
print("="*80)

results = [
    ['Model', 'Variable', 'β', 'SE', 'p', 'R²_adj', 'DW'],
    ['─'*12, '─'*23, '─'*8, '─'*8, '─'*8, '─'*8, '─'*6],
    ['Model 1', 'Population density', 
     f"{model1.params.iloc[1]:.3f}", 
     f"{model1.bse.iloc[1]:.3f}", 
     f"{model1.pvalues.iloc[1]:.3f}",
     f"{model1.rsquared_adj:.3f}",
     f"{durbin_watson(model1.resid):.2f}"],
    ['', '', '', '', '', '', ''],
    ['Model 2', 'Population density',
     f"{model2.params.iloc[1]:.3f}",
     f"{model2.bse.iloc[1]:.3f}",
     f"{model2.pvalues.iloc[1]:.3f}",
     f"{model2.rsquared_adj:.3f}",
     f"{durbin_watson(model2.resid):.2f}"],
    ['', 'D_spatial',
     f"{model2.params.iloc[2]:.3f}",
     f"{model2.bse.iloc[2]:.3f}",
     f"{model2.pvalues.iloc[2]:.3f}",
     '', ''],
    ['', '', '', '', '', '', ''],
    ['Model 3', 'Population density',
     f"{model3.params.iloc[1]:.3f}",
     f"{model3.bse.iloc[1]:.3f}",
     f"{model3.pvalues.iloc[1]:.3f}",
     f"{model3.rsquared_adj:.3f}",
     f"{durbin_watson(model3.resid):.2f}"],
    ['', 'D_knowledge',
     f"{model3.params.iloc[2]:.3f}",
     f"{model3.bse.iloc[2]:.3f}",
     f"{model3.pvalues.iloc[2]:.3f}",
     '', ''],
    ['', 'D_support',
     f"{model3.params.iloc[3]:.3f}",
     f"{model3.bse.iloc[3]:.3f}",
     f"{model3.pvalues.iloc[3]:.3f}",
     '', ''],
]

for row in results:
    print(f"{row[0]:<12} {row[1]:<23} {row[2]:>8} {row[3]:>8} {row[4]:>8} {row[5]:>8} {row[6]:>6}")


#Table to compare with paper results:

print("\n" + "="*80)
print("COMPARISON WITH PAPER (TABLE 1)")
print("="*80)

comparisons = [
    ['Variable', 'Paper β', 'Our β', 'Diff', 'Match'],
    ['─'*25, '─'*8, '─'*8, '─'*8, '─'*5],
    ['Pop density (Model 1)', '0.636', f"{model1.params.iloc[1]:.3f}", 
     f"{abs(0.636 - model1.params.iloc[1]):.3f}", 
     '✓' if abs(0.636 - model1.params.iloc[1]) < 0.10 else '✗'],
    ['Pop density (Model 2)', '0.565', f"{model2.params.iloc[1]:.3f}",
     f"{abs(0.565 - model2.params.iloc[1]):.3f}",
     '✓' if abs(0.565 - model2.params.iloc[1]) < 0.10 else '✗'],
    ['D_spatial (Model 2)', '0.243', f"{model2.params.iloc[2]:.3f}",
     f"{abs(0.243 - model2.params.iloc[2]):.3f}",
     '✓' if abs(0.243 - model2.params.iloc[2]) < 0.10 else '✗'],
    ['Pop density (Model 3)', '0.471', f"{model3.params.iloc[1]:.3f}",
     f"{abs(0.471 - model3.params.iloc[1]):.3f}",
     '✓' if abs(0.471 - model3.params.iloc[1]) < 0.10 else '✗'],
    ['D_knowledge (Model 3)', '1.033', f"{model3.params.iloc[2]:.3f}",
     f"{abs(1.033 - model3.params.iloc[2]):.3f}",
     '✓' if abs(1.033 - model3.params.iloc[2]) < 0.10 else '✗'],
    ['D_support (Model 3)', '-0.555', f"{model3.params.iloc[3]:.3f}",
     f"{abs(-0.555 - model3.params.iloc[3]):.3f}",
     '✓' if abs(-0.555 - model3.params.iloc[3]) < 0.10 else '✗'],
]

for row in comparisons:
    print(f"{row[0]:<25} {row[1]:>8} {row[2]:>8} {row[3]:>8} {row[4]:>5}")

print("\n" + "="*80)
print("CONFIGURATION USED:")
print("  - Full graph diversity: diversity_spatial_all_minstrength_5")
print("  - Knowledge diversity: spatial_diversity_knowledge_0.99")
print("  - Support diversity: spatial_diversity_support_0.99")
print("="*80)

#full regression summaries
print("\n" + "="*80)
print("DETAILED REGRESSION OUTPUTS")
print("="*80)

print("\n--- MODEL 1: Population Density Only ---")
print(model1.summary())

print("\n--- MODEL 2: Full Communication Graph ---")
print(model2.summary())

print("\n--- MODEL 3: Dimension-Specific Graphs ---")
print(model3.summary())

TABLE 1 REPRODUCTION RESULTS
Model        Variable                       β       SE        p   R²_adj     DW
──────────── ─────────────────────── ──────── ──────── ──────── ──────── ──────
Model 1      Population density         0.548    0.129    0.000    0.283   1.96
                                                                               
Model 2      Population density         0.492    0.139    0.001    0.286   2.02
             D_spatial                  0.148    0.139    0.293                
                                                                               
Model 3      Population density         0.400    0.097    0.000    0.623   2.05
             D_knowledge                0.842    0.136    0.000                
             D_support                 -0.467    0.133    0.001                

COMPARISON WITH PAPER (TABLE 1)
Variable                   Paper β    Our β     Diff Match
───────────────────────── ──────── ──────── ──────── ─────
Pop density (Model 1

## First approach ##
However, our first approach was to manually compute spatial diversity measures following equations (6-11) from the paper. We implemented the following workflow:
- Step 1: Load edge-level data from Reddit messages
- Step 2: For each user, calculate spatial diversity (normalized entropy of destination states)
- Step 3: Aggregate user-level diversities to state level by computing means
- Step 4: Run three regression models predicting GDP per capita

This approach involved:
- Computing user-level spatial diversity for all edges (full communication graph)
- Computing dimension-specific diversities by filtering edges where knowledge_binary_adaptive_0.99 == 1 or support_binary_adaptive_0.99 == 1
- Aggregating to state level using mean diversity across users in each state.

But as showed in the final Table after the following code, the results were differents from the paper.

In [3]:
# utility to compute user-level diversity given a user->state weight table
def compute_user_diversities(user_state_weights_df):
    """
    user_state_weights_df: DataFrame with columns ['author', 'author_state_code', 'dest_state_code', 'weight']
    returns: DataFrame [author, author_state_code, D_spatial_user]
    """
    rows = []
    # group by author
    for author, grp in user_state_weights_df.groupby("author"):
        src_state = grp["author_state_code"].iloc[0]
        weights = grp["weight"].values.astype(float)
        total = weights.sum()
        if total <= 0 or len(weights) <= 1:
            D = 0.0
        else:
            p = weights / total
            H = -np.sum(p * np.log(p))
            # normalize by log(n_dest)
            n_dest = len(weights)
            H_norm = H / np.log(n_dest) if n_dest > 1 else 0.0
            # paper uses "1 - normalized_entropy"
            D = 1.0 - H_norm
        rows.append((author, src_state, D))
    return pd.DataFrame(rows, columns=["author", "author_state_code", "D_spatial_user"])

# --- 1) Full graph: build user->dest_state counts (all messages)
user_state_all = (
    dim_df.groupby(["author", "author_state_code", "dest_state_code"])
    .size()
    .reset_index(name="weight")
)

# compute per-user diversity (full graph)
user_div_all = compute_user_diversities(user_state_all)

# 2) Knowledge graph: build user->dest_state counts only for knowledge messages
col_k = "knowledge_binary_adaptive_0.99"  # adapt if your column name differs
user_state_know = (
    dim_df[dim_df[col_k] == 1]
    .groupby(["author", "author_state_code", "dest_state_code"])
    .size()
    .reset_index(name="weight")
)
user_div_know = compute_user_diversities(user_state_know)

# 3) Support graph
col_s = "support_binary_adaptive_0.99"
user_state_support = (
    dim_df[dim_df[col_s] == 1]
    .groupby(["author", "author_state_code", "dest_state_code"])
    .size()
    .reset_index(name="weight")
)
user_div_support = compute_user_diversities(user_state_support)

# 4) Aggregate to state level by averaging user-level D

D_all_state = user_div_all.groupby("author_state_code")["D_spatial_user"].mean().reset_index().rename(
    columns={"author_state_code":"state_code", "D_spatial_user":"D_spatial_all"}
)

D_know_state = user_div_know.groupby("author_state_code")["D_spatial_user"].mean().reset_index().rename(
    columns={"author_state_code":"state_code", "D_spatial_user":"D_spatial_knowledge"}
)

D_support_state = user_div_support.groupby("author_state_code")["D_spatial_user"].mean().reset_index().rename(
    columns={"author_state_code":"state_code", "D_spatial_user":"D_spatial_support"}
)

# merge with df
reg = df.copy()  # df is your regression file loaded at top of notebook
reg = reg.merge(D_all_state, on="state_code", how="left")
reg = reg.merge(D_know_state, on="state_code", how="left")
reg = reg.merge(D_support_state, on="state_code", how="left")

In [6]:
# Standardize and run regressions
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson

def standardized_ols_df(y, X_df):
    scaler = StandardScaler()
    X_std = pd.DataFrame(scaler.fit_transform(X_df), columns=X_df.columns, index=X_df.index)
    y_std = (y - y.mean()) / y.std()
    X_std = sm.add_constant(X_std)
    model = sm.OLS(y_std, X_std).fit(cov_type="HC3")  # robust SE
    return model

y = reg["gdp per capita (2017)"]

X_a = reg[["population_density"]]
X_b = reg[["population_density", "D_spatial_all"]]
X_c = reg[["population_density", "D_spatial_knowledge", "D_spatial_support"]]

m1 = standardized_ols_df(y, X_a)
m2 = standardized_ols_df(y, X_b)
m3 = standardized_ols_df(y, X_c)

def summarize_model(name, model):
    vars = [v for v in model.params.index if v != "const"]
    print(f"\nModel: {name}")
    for v in vars:
        print(f"{v:30s} β={model.params[v]:.3f}  SE={model.bse[v]:.3f}  p={model.pvalues[v]:.3f}")
    print(f"Adj R2 = {model.rsquared_adj:.3f}, Durbin-Watson = {durbin_watson(model.resid):.3f}")

summarize_model("Baseline", m1)
summarize_model("Full graph", m2)
summarize_model("Dim-specific", m3)


Model: Baseline
population_density             β=0.542  SE=0.195  p=0.005
Adj R2 = 0.283, Durbin-Watson = 1.963

Model: Full graph
population_density             β=0.549  SE=0.159  p=0.001
D_spatial_all                  β=0.353  SE=0.112  p=0.002
Adj R2 = 0.400, Durbin-Watson = 2.025

Model: Dim-specific
population_density             β=0.495  SE=0.177  p=0.005
D_spatial_knowledge            β=0.300  SE=0.202  p=0.138
D_spatial_support              β=0.107  SE=0.239  p=0.653
Adj R2 = 0.397, Durbin-Watson = 1.971


In [7]:
#Results summary:

print("="*80)
print("TABLE 1 REPRODUCTION RESULTS")
print("="*80)

results = [
    ['Model', 'Variable', 'β', 'SE', 'p', 'R²_adj', 'DW'],
    ['─'*12, '─'*23, '─'*8, '─'*8, '─'*8, '─'*8, '─'*6],
    ['Model 1', 'Population density', 
     f"{m1.params.iloc[1]:.3f}", 
     f"{m1.bse.iloc[1]:.3f}", 
     f"{m1.pvalues.iloc[1]:.3f}",
     f"{m1.rsquared_adj:.3f}",
     f"{durbin_watson(m1.resid):.2f}"],
    ['', '', '', '', '', '', ''],
    ['Model 2', 'Population density',
     f"{m2.params.iloc[1]:.3f}",
     f"{m2.bse.iloc[1]:.3f}",
     f"{m2.pvalues.iloc[1]:.3f}",
     f"{m2.rsquared_adj:.3f}",
     f"{durbin_watson(m2.resid):.2f}"],
    ['', 'D_spatial',
     f"{m2.params.iloc[2]:.3f}",
     f"{m2.bse.iloc[2]:.3f}",
     f"{m2.pvalues.iloc[2]:.3f}",
     '', ''],
    ['', '', '', '', '', '', ''],
    ['Model 3', 'Population density',
     f"{m3.params.iloc[1]:.3f}",
     f"{m3.bse.iloc[1]:.3f}",
     f"{m3.pvalues.iloc[1]:.3f}",
     f"{m3.rsquared_adj:.3f}",
     f"{durbin_watson(m3.resid):.2f}"],
    ['', 'D_knowledge',
     f"{m3.params.iloc[2]:.3f}",
     f"{m3.bse.iloc[2]:.3f}",
     f"{m3.pvalues.iloc[2]:.3f}",
     '', ''],
    ['', 'D_support',
     f"{m3.params.iloc[3]:.3f}",
     f"{m3.bse.iloc[3]:.3f}",
     f"{m3.pvalues.iloc[3]:.3f}",
     '', ''],
]

for row in results:
    print(f"{row[0]:<12} {row[1]:<23} {row[2]:>8} {row[3]:>8} {row[4]:>8} {row[5]:>8} {row[6]:>6}")


#Table to compare with paper results:

print("\n" + "="*80)
print("COMPARISON WITH PAPER (TABLE 1)")
print("="*80)

comparisons = [
    ['Variable', 'Paper β', 'Our β', 'Diff', 'Match'],
    ['─'*25, '─'*8, '─'*8, '─'*8, '─'*5],
    ['Pop density (Model 1)', '0.636', f"{m1.params.iloc[1]:.3f}", 
     f"{abs(0.636 - m1.params.iloc[1]):.3f}", 
     '✓' if abs(0.636 - m1.params.iloc[1]) < 0.10 else '✗'],
    ['Pop density (Model 2)', '0.565', f"{m2.params.iloc[1]:.3f}",
     f"{abs(0.565 - m2.params.iloc[1]):.3f}",
     '✓' if abs(0.565 - m2.params.iloc[1]) < 0.10 else '✗'],
    ['D_spatial (Model 2)', '0.243', f"{m2.params.iloc[2]:.3f}",
     f"{abs(0.243 - m2.params.iloc[2]):.3f}",
     '✓' if abs(0.243 - m2.params.iloc[2]) < 0.10 else '✗'],
    ['Pop density (Model 3)', '0.471', f"{m3.params.iloc[1]:.3f}",
     f"{abs(0.471 - m3.params.iloc[1]):.3f}",
     '✓' if abs(0.471 - m3.params.iloc[1]) < 0.10 else '✗'],
    ['D_knowledge (Model 3)', '1.033', f"{m3.params.iloc[2]:.3f}",
     f"{abs(1.033 - m3.params.iloc[2]):.3f}",
     '✓' if abs(1.033 - m3.params.iloc[2]) < 0.10 else '✗'],
    ['D_support (Model 3)', '-0.555', f"{m3.params.iloc[3]:.3f}",
     f"{abs(-0.555 - m3.params.iloc[3]):.3f}",
     '✓' if abs(-0.555 - m3.params.iloc[3]) < 0.10 else '✗'],
]

for row in comparisons:
    print(f"{row[0]:<25} {row[1]:>8} {row[2]:>8} {row[3]:>8} {row[4]:>5}")

TABLE 1 REPRODUCTION RESULTS
Model        Variable                       β       SE        p   R²_adj     DW
──────────── ─────────────────────── ──────── ──────── ──────── ──────── ──────
Model 1      Population density         0.542    0.195    0.005    0.283   1.96
                                                                               
Model 2      Population density         0.549    0.159    0.001    0.400   2.02
             D_spatial                  0.353    0.112    0.002                
                                                                               
Model 3      Population density         0.495    0.177    0.005    0.397   1.97
             D_knowledge                0.300    0.202    0.138                
             D_support                  0.107    0.239    0.653                

COMPARISON WITH PAPER (TABLE 1)
Variable                   Paper β    Our β     Diff Match
───────────────────────── ──────── ──────── ──────── ─────
Pop density (Model 1